# Spell-checking

Checando manualmente los datos hay muchos que tienen errores, lo que es de esperar pues la mayoría vienen de Twitter. Sería conveniente aplicar un corrector:

-  [contextualSpellCheck](https://spacy.io/universe/project/contextualSpellCheck), de SpaCy, se ve bien ya que usa Bert, pero cambia demasiado los nombres propios y otras entidades por sus respectivos pronombres. Además, borra ciertas groserias y otras palabras que pueden ser importantes para esta tarea.
- [Autocorrect](https://github.com/fsondej/autocorrect) se ve bien aunque es más sencillo, si no reconoce algo lo deja como está, es probable que se me vayan más errores ortográficos con esta librería. Sin embargo, también borra ciertas groserias y otras palabras que pueden ser importantes para esta tarea. 
- [pyspellchecker](https://github.com/barrust/pyspellchecker) también se ve bien. A diferencia de las anteriores, deja las groserias y otras palabras que pueden ser importantes para esta tarea intactas, de modo que me quedaré con este.

El inconveniente es que es un poco tardado hacer este spell-check.

In [1]:
import pandas as pd
import re
from tqdm import tqdm, trange

from utils import preprocessing

In [2]:
text = "Bitches be begging me to follw them just to give me a reason not to follw them. Lol @david_guzman"
%time preprocessing.spell_check(text, language='en')

CPU times: user 653 µs, sys: 0 ns, total: 653 µs
Wall time: 656 µs


'Bitches be begging me to follow them just to give me a reason not to follow them . Lol @ david _ guzman'

In [3]:
text = "Las putass me ruegan que las sigaa solo para darme una razón para no seguirlass. Lol @david_guzman"
preprocessing.spell_check(text, language='es')

'Las putas me ruegan que las siga solo para darme una razón para no seguirlas . Lol @ david _ gustan'

## Train

In [4]:
train_df = pd.read_csv('../../Data/EXIST2021_training.tsv', sep='\t')

# Un simple pre-procesamiento
train_df['text'] = train_df['text'].apply(lambda text: preprocessing.preprocess(text))

train_df.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,"she calls herself "" anti-feminazi "" how about ...",sexist,ideological-inequality
1,EXIST2021,2,twitter,en,"now , back to these women , the brave and the ...",non-sexist,non-sexist
2,EXIST2021,3,twitter,en,"wow , your skirt is very short . what is it's ...",sexist,objectification
3,EXIST2021,4,twitter,en,incredible ! beautiful ! but i laughed so much...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist


In [6]:
corrected_text = []

for value in tqdm(train_df[['language', 'text']].values):
    language, text = value
    
    corrected_text.append(preprocessing.spell_check(text, language=language))

100%|██████████| 6977/6977 [19:42<00:00,  5.90it/s] 


In [7]:
train_df['text'] = corrected_text
train_df.to_csv('../../Data/EXIST2021_training_spell_checked.csv', index=False)

## Test

In [8]:
test_df = pd.read_csv('../../Data/EXIST2021_test.tsv', sep='\t')

# Un simple pre-procesamiento
test_df['text'] = test_df['text'].apply(lambda text: preprocessing.preprocess(text))

test_df.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,6978,gab,en,pennsylvania state rep horrifies with opening ...,non-sexist,non-sexist
1,EXIST2021,6979,twitter,en,"he sounds like as ass , and very condescending .",non-sexist,non-sexist
2,EXIST2021,6980,twitter,en,"lol ! "" this behavior of not letting men tell ...",sexist,ideological-inequality
3,EXIST2021,6981,twitter,en,rights ? i mean yeah most women especially the...,sexist,ideological-inequality
4,EXIST2021,6982,twitter,en,the jack manifold appreciation i ’ m seeing is...,non-sexist,non-sexist


In [9]:
corrected_text = []

for value in tqdm(test_df[['language', 'text']].values):
    language, text = value
    
    corrected_text.append(preprocessing.spell_check(text, language=language))

100%|██████████| 4368/4368 [11:37<00:00,  6.26it/s]


In [10]:
test_df['text'] = corrected_text
test_df.to_csv('../../Data/EXIST2021_test_spell_checked.csv', index=False)